In [1]:
import pandas as pd
import numpy as np

adf = pd.read_csv("../legislator_info.csv")
adf = adf.drop(["Unnamed: 0","API URL"],axis=1)
#suffix_list = ["Jr.","Sr.","IV","II","III"]

# Extract Suffix and remove from source column
adf['Suffix'] = adf['Name'].str.extract(r'(Jr\.|III|IV|II|Sr\.)')

# Replace suffixes with empty strings in original source column (so it isn't pulled again)
adf['Name'] = adf['Name'].str.replace(r'(Jr\.|III|IV|II|Sr\.)', '',regex=True)

# Extract nicknames and remove from source column (so it isn't pulled again)
adf["Nickname"] = adf["Name"].str.extract(r'("[^"]+"|\([^)]+\))')
adf["Name"] = adf["Name"].str.replace(r'("[^"]+"|\([^)]+\))',"",regex=True)

# Split name into separate columns
adf[["Last","First_temp","Suffix_temp"]] = adf["Name"].str.split(",",expand=True)

# Remove extra characters and spaces from Nickname
adf["Nickname"] = adf["Nickname"].str.replace("(","",regex=False).str.replace(")", "",regex=False).str.replace('"', "",regex=False).str.replace(",","",regex=False)

# If you need to check a specific row
#adf.loc[779]


In [2]:
# Verify Suffix looks good
adf["Suffix"].value_counts()

Jr.    107
III     10
IV       4
II       4
Sr.      3
Name: Suffix, dtype: int64

In [3]:
# Verify Nickname looks good
adf["Nickname"].value_counts()

Buddy           3
Bill            3
Gene            2
Skip            1
Joe             1
Bob             1
Dick            1
Dee             1
Jake            1
Randy           1
Butch           1
Buck            1
Bud             1
Dan             1
Duke            1
Jack            1
Mrs. Gillis     1
Jim             1
Jerry           1
Al              1
Lindy           1
Sonny           1
Pete            1
Quico           1
Chuck           1
Rick            1
Hank            1
Joseph          1
Jimmy           1
Stan            1
Chuy            1
Chip            1
Jim Guy         1
Billy           1
Robin           1
Mrs. John B.    1
Bobby           1
Fred            1
Name: Nickname, dtype: int64

In [4]:
# Split First_temp based on space and store in temp df
name_split = adf["First_temp"].str.split(" ",expand=True)
#name_split.to_csv("test.csv")

name_split

,0,1,2,3,4
0,,Ronald,A.,None,None
1,,Ted,None,None,None
2,,Elizabeth,None,None,None
3,,Sheldon,None,None,None
4,,Todd,None,None,None
...,...,...,...,...,...
2495,,Bob,None,None,None
2496,,William,W.,None,None
2497,,Barney,None,None,None
2498,,Jon,D.,None,None


In [6]:
# Check different columns of name_split to see what it contains
# Should hopefully only need first three worth
name_split.loc[name_split[3].notnull(),3].tolist()

['',
 'Q.',
 '',
 '',
 '',
 'Coleman',
 '',
 'S.',
 'R.',
 '',
 'Camacho',
 '',
 'J.',
 '',
 '',
 '',
 'W.',
 'Bill',
 '',
 '',
 'F.',
 '',
 '',
 'H.',
 '',
 'F.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'C.',
 '',
 '',
 'Dutch',
 '',
 '',
 'Happy',
 'H.',
 'Bryan',
 '',
 '',
 '',
 '',
 '',
 '',
 'B.']

In [7]:
# After confirming name_split looks good, bring into actuals
adf["First_Actual"] = name_split[1]
adf["Middle_1"] = name_split[2]
adf["Middle_2"] = name_split[3]

In [8]:
# Confirm Suffix is empty before dropping
adf["Suffix_temp"].value_counts()

      119
        4
        1
Name: Suffix_temp, dtype: int64

In [9]:
# Clean up columns and empty strings
adf = adf.drop(["First_temp","Suffix_temp",],axis=1)
adf = adf.replace("",np.nan)

In [14]:
# Run these two to make sure Middle_1 and Middle_2 look good
adf.loc[adf["Middle_1"].notnull(),"Middle_1"].tolist()
adf["Middle_2"].value_counts()

F.         2
H.         2
Q.         1
Coleman    1
S.         1
R.         1
Camacho    1
J.         1
W.         1
Bill       1
C.         1
Dutch      1
Happy      1
Bryan      1
B.         1
Name: Middle_2, dtype: int64

In [ ]:
# Export 
adf.to_csv("test2.csv")